In [143]:
import newspaper
from newsapi import NewsApiClient
from datetime import date
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

import spacy
nlp = spacy.load('en_core_web_lg')

In [26]:
api_key = 'eeeaefaae3c14737bc08e252a6e1991b'
newsapi = NewsApiClient(api_key=api_key)

In [133]:
def get_trending_articles_today():
    today = date.today().strftime("%Y-%m-%d")
    trending_topics = newspaper.hot()[:5]

    data = []
    for topic in trending_topics:
        articles = []
        for i in range(1, 5):
            page_articles = newsapi.get_everything(q=topic,
                                      language='en',
                                      from_param=today,
                                      page=i)

            if len(page_articles) == 0:
                break
            else:
                articles.extend(page_articles["articles"])

        # add article info
        article_info = [(article['publishedAt'], article['title'], article['url'], topic) for article in articles]
        data.extend(article_info)
    
    return pd.DataFrame(data, columns=["date", "title", "url", "topic"])

In [197]:
df = get_trending_articles_today()
df = df.drop_duplicates(subset=['title'])

In [205]:
sent_vecs = {}
for title in df.loc[df.topic == "NASCAR"].title:
    try:
        doc = nlp(title)
        sent_vecs.update({'title': doc.vector})
    except Exception as e:
        print(e)

vectors = list(sent_vecs.values())
x = np.array(vectors)
dbscan = DBSCAN(eps=100, min_samples=2, metric='cosine').fit(x)

In [206]:
dbscan.labels_

array([-1], dtype=int64)